<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_1000rep_n30_x2_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_1000rep_n30_x2_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...       y_hat        e_hat
rep   index                        ...                         
0.0   0.0    48.903885  40.546803  ...  624.173460  2210.060625
      1.0    16.325213  17.500183  ...  212.803786  -308.825459
      2.0    20.578167  23.674307  ...  244.194160   491.969173
      3.0    12.329733  28.225962  ...  172.369817   -15.788135
      4.0    18.612859  22.604111  ...  238.075632  -236.118260
...                ...        ...  ...         ...          ...
999.0 25.0   49.764443  30.138338  ...  604.343664 -6926.270010
      26.0   28.291565  28.004700  ...  355.165083   -74.911947
      27.0   40.218787  45.732215  ...  526.446340  1003.103239
      28.0   21.161301  48.267915  ...  335.886659  -109.692912
      29.0   31.801785  48.190311  ...  454.409054  -310.991105

[30000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...       y_hat        e_hat
rep   index                        ...                         
0.0   0.0    48.903885  40.546803  ...  624.173460  2210.060625
      1.0    16.325213  17.500183  ...  212.803786  -308.825459
      2.0    20.578167  23.674307  ...  244.194160   491.969173
      3.0    12.329733  28.225962  ...  172.369817   -15.788135
      4.0    18.612859  22.604111  ...  238.075632  -236.118260
...                ...        ...  ...         ...          ...
999.0 25.0   49.764443  30.138338  ...  604.343664 -6926.270010
      26.0   28.291565  28.004700  ...  355.165083   -74.911947
      27.0   40.218787  45.732215  ...  526.446340  1003.103239
      28.0   21.161301  48.267915  ...  335.886659  -109.692912
      29.0   31.801785  48.190311  ...  454.409054  -310.991105

[30000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,48.903885,40.546803,49.750707,2249.897727,2834.234085,624.173460,2210.060625
1,0.0,1.0,16.325213,17.500183,41.532079,-323.306064,-96.021673,212.803786,-308.825459
2,0.0,2.0,20.578167,23.674307,11.163827,490.543525,736.163333,244.194160,491.969173
3,0.0,3.0,12.329733,28.225962,19.678308,-19.619916,156.581682,172.369817,-15.788135
4,0.0,4.0,18.612859,22.604111,30.074844,-241.850172,1.957372,238.075632,-236.118260
...,...,...,...,...,...,...,...,...,...
29995,999.0,25.0,49.764443,30.138338,42.604168,-6897.313285,-6321.926347,604.343664,-6926.270010
29996,999.0,26.0,28.291565,28.004700,30.370993,-66.038202,280.253136,355.165083,-74.911947
29997,999.0,27.0,40.218787,45.732215,33.976324,1042.653173,1529.549579,526.446340,1003.103239
29998,999.0,28.0,21.161301,48.267915,45.750484,-84.437657,226.193747,335.886659,-109.692912


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...       y_hat        e_hat
rep   index                        ...                         
0.0   0.0    48.903885  40.546803  ...  624.173460  2210.060625
      1.0    16.325213  17.500183  ...  212.803786  -308.825459
      2.0    20.578167  23.674307  ...  244.194160   491.969173
      3.0    12.329733  28.225962  ...  172.369817   -15.788135
      4.0    18.612859  22.604111  ...  238.075632  -236.118260
...                ...        ...  ...         ...          ...
999.0 25.0   49.764443  30.138338  ...  604.343664 -6926.270010
      26.0   28.291565  28.004700  ...  355.165083   -74.911947
      27.0   40.218787  45.732215  ...  526.446340  1003.103239
      28.0   21.161301  48.267915  ...  335.886659  -109.692912
      29.0   31.801785  48.190311  ...  454.409054  -310.991105

[30000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
0.0,15.0,9.0,2092437.0440834675,79506950.5866137,149459.78886310483,9938368.823326712,63.32882735263644,5.270062164441924e-10,0.9999999997364969,Reject005=0 : Heteroscedasticity
2.0,10.0,14.0,14364003.239103595,28824379.567438696,1596000.3599003993,2217259.9667260535,1.4333638196433163,0.57427413916822,0.71286293041589,Reject005=1 : Homoscedasticity
3.0,12.0,12.0,4864995.326406568,28478484.48397849,442272.30240059714,2588953.1349071357,5.853753718816737,0.00459074400128312,0.9977046279993584,Reject005=0 : Heteroscedasticity
5.0,14.0,10.0,10273600.65917013,11250858.018999502,790276.9737823177,1250095.3354443891,1.5331724240750895,0.4512491442343687,0.7743754278828157,Reject005=1 : Homoscedasticity
6.0,11.0,13.0,4116951.191246464,34134760.9870634,411695.1191246464,2844563.415588617,7.01569145589048,0.0027076207646550277,0.9986461896176725,Reject005=0 : Heteroscedasticity
8.0,13.0,11.0,3256764.847287334,28377397.030137457,271397.07060727786,2837739.7030137456,10.297619059240251,0.00020476038206829195,0.9998976198089659,Reject005=0 : Heteroscedasticity
9.0,14.0,10.0,6253463.1618104195,21056030.26630543,481035.62783157075,2339558.9184783814,4.713938758422845,0.008962553472297463,0.9955187232638513,Reject005=0 : Heteroscedasticity
10.0,11.0,13.0,2043141.715049589,55972122.24278775,204314.1715049589,4664343.520232312,23.18049020499716,8.294086306648296e-06,0.9999958529568467,Reject005=0 : Heteroscedasticity
16.0,13.0,11.0,1145958.5989241963,16412094.479324972,95496.5499103497,1641209.4479324971,16.925665268878607,1.2391591146299064e-05,0.9999938042044269,Reject005=0 : Heteroscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,15.0,9.0,2092437.0440834675,79506950.5866137,149459.78886310483,9938368.823326712,63.32882735263644,5.270062164441924e-10,Reject005=0 : Heteroscedasticity
1.0,12.0,12.0,12935931.60765359,3758674.5419718293,1175993.7825139628,341697.68563380267,0.29056079267982493,0.041708941101022225,Reject005=0 : Heteroscedasticity
2.0,10.0,14.0,14364003.239103595,28824379.567438696,1596000.3599003993,2217259.9667260535,1.4333638196433163,1.42572586083178,Reject005=1 : Homoscedasticity
3.0,12.0,12.0,4864995.326406568,28478484.48397849,442272.30240059714,2588953.1349071357,5.853753718816737,0.00459074400128312,Reject005=0 : Heteroscedasticity
4.0,16.0,8.0,7389727.139382287,2787405.1079254653,492648.47595881915,398200.7297036379,0.7544000083766196,0.7084456958360323,Reject005=1 : Homoscedasticity
...,...,...,...,...,...,...,...,...,...
995.0,16.0,8.0,23256605.73656993,20993138.05368135,1550440.3824379954,2999019.721954479,1.8053484065106387,0.2994001448347019,Reject005=1 : Homoscedasticity
996.0,12.0,12.0,9802654.507818012,39185021.84037695,891150.4098016374,3562274.712761541,3.997388851063283,0.02336921976925921,Reject005=0 : Heteroscedasticity
997.0,13.0,11.0,24085993.62639993,12671721.848982183,2007166.1355333275,1267172.1848982184,0.6217585003284013,0.4364230275410335,Reject005=1 : Homoscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    557
Reject005=1 : Homoscedasticity      443
Name: Result_test, dtype: int64